In [1]:
import os
import re
import traceback
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox

# ---------- 1) CORE FUNCTION: Build file index ----------
def build_file_index(base_dir: str) -> pd.DataFrame:
    rows = []

    for root, _, files in os.walk(base_dir):
        for f in files:
            f_low = f.lower()

            # Only transaction files
            if "datatrans" not in f_low:
                continue
            if not f_low.endswith((".csv", ".xlsx")):
                continue

            filepath = os.path.join(root, f)

            # Robust parsing: expects .../<YEAR>/<MONTH>/<DD-MM-YY>/DataTrans_...
            parts = os.path.normpath(filepath).split(os.sep)

            year = None
            month_folder = None
            statement_folder = None
            statement_date = None

            if len(parts) >= 4:
                year_str = parts[-4]
                month_folder = parts[-3]
                statement_folder = parts[-2]

                if year_str.isdigit():
                    year = int(year_str)

                m = re.match(r"(\d{2})-(\d{2})-(\d{2})$", statement_folder)
                if m:
                    dd, mm, yy = map(int, m.groups())
                    statement_date = datetime(2000 + yy, mm, dd).date()

            rows.append({
                "filepath": filepath,
                "year": year,
                "month_folder": month_folder,
                "statement_folder": statement_folder,
                "statement_date": statement_date
            })

    df = pd.DataFrame(rows)
    return df


# ---------- 2) GUI ----------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("FP&A Report Generator (Task 4)")
        self.geometry("700x260")
        self.resizable(False, False)

        self.selected_folder = tk.StringVar(value="No folder selected yet.")
        self.status = tk.StringVar(value="Step 1: Click 'Pick Transactions Folder'")

        tk.Label(self, text="Transactions Folder (contains 2024/2025):").pack(anchor="w", padx=12, pady=(12, 2))
        tk.Entry(self, textvariable=self.selected_folder, width=95).pack(padx=12)

        tk.Button(self, text="Pick Transactions Folder", command=self.pick_folder, height=2).pack(pady=12)

        tk.Label(self, textvariable=self.status, fg="navy", wraplength=660, justify="left").pack(padx=12, pady=8)

    def pick_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select folder that contains 2024/2025")
            if not folder:
                self.status.set("No folder selected.")
                return

            self.selected_folder.set(folder)
            self.status.set("Scanning files...")
            self.update_idletasks()

            df = build_file_index(folder)

            years = sorted(df["year"].dropna().unique())
            self.status.set(f"✅ Files found: {len(df)} | Years detected: {years}")

            messagebox.showinfo("Scan Complete", f"Files found: {len(df)}\nYears detected: {years}")

        except Exception:
            self.status.set("❌ Error. See console for details.")
            print(traceback.format_exc())


if __name__ == "__main__":
    App().mainloop()


In [2]:
python task4_gui.py


SyntaxError: invalid syntax (4204781267.py, line 1)

In [3]:
import os
import re
import traceback
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox

# ---------- 1) CORE FUNCTION: Build file index ----------
def build_file_index(base_dir: str) -> pd.DataFrame:
    rows = []

    for root, _, files in os.walk(base_dir):
        for f in files:
            f_low = f.lower()

            # Only transaction files
            if "datatrans" not in f_low:
                continue
            if not f_low.endswith((".csv", ".xlsx")):
                continue

            filepath = os.path.join(root, f)

            # Robust parsing: expects .../<YEAR>/<MONTH>/<DD-MM-YY>/DataTrans_...
            parts = os.path.normpath(filepath).split(os.sep)

            year = None
            month_folder = None
            statement_folder = None
            statement_date = None

            if len(parts) >= 4:
                year_str = parts[-4]
                month_folder = parts[-3]
                statement_folder = parts[-2]

                if year_str.isdigit():
                    year = int(year_str)

                m = re.match(r"(\d{2})-(\d{2})-(\d{2})$", statement_folder)
                if m:
                    dd, mm, yy = map(int, m.groups())
                    statement_date = datetime(2000 + yy, mm, dd).date()

            rows.append({
                "filepath": filepath,
                "filetype": os.path.splitext(filepath)[1].lower().replace(".", ""),
                "year": year,
                "month_folder": month_folder,
                "statement_folder": statement_folder,
                "statement_date": statement_date
            })

    df = pd.DataFrame(rows)

    # Optional: convert dates if present
    if "statement_date" in df.columns:
        df["statement_date"] = pd.to_datetime(df["statement_date"], errors="coerce")

    return df


# ---------- 2) GUI ----------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("FP&A Report Generator (Task 4)")
        self.geometry("700x260")
        self.resizable(False, False)

        self.selected_folder = tk.StringVar(value="No folder selected yet.")
        self.status = tk.StringVar(value="Step 1: Click 'Pick Transactions Folder'")

        tk.Label(self, text="Transactions Folder (contains 2024/2025):").pack(anchor="w", padx=12, pady=(12, 2))
        tk.Entry(self, textvariable=self.selected_folder, width=95).pack(padx=12)

        tk.Button(self, text="Pick Transactions Folder", command=self.pick_folder, height=2).pack(pady=12)

        tk.Label(self, textvariable=self.status, fg="navy", wraplength=660, justify="left").pack(padx=12, pady=8)

    def pick_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select folder that contains 2024/2025")
            if not folder:
                self.status.set("No folder selected.")
                return

            self.selected_folder.set(folder)
            self.status.set("Scanning files...")
            self.update_idletasks()

            df = build_file_index(folder)

            # ✅ Clean years (convert numpy ints to plain Python int)
            years = sorted(int(y) for y in df["year"].dropna().unique())

            self.status.set(f"✅ Files found: {len(df)} | Years detected: {years}")

            messagebox.showinfo(
                "Scan Complete",
                f"Files found: {len(df)}\nYears detected: {years}"
            )

        except Exception:
            self.status.set("❌ Error. See console for details.")
            print(traceback.format_exc())


if __name__ == "__main__":
    App().mainloop()


In [5]:
import os
import re
import traceback
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox

# -----------------------------
# 1) Build file index (your working logic)
# -----------------------------
def build_file_index(base_dir: str) -> pd.DataFrame:
    rows = []

    for root, _, files in os.walk(base_dir):
        for f in files:
            f_low = f.lower()

            if "datatrans" not in f_low:
                continue
            if not f_low.endswith((".csv", ".xlsx")):
                continue

            filepath = os.path.join(root, f)

            # expects .../<YEAR>/<MONTH>/<DD-MM-YY>/DataTrans_...
            parts = os.path.normpath(filepath).split(os.sep)

            year = None
            month_folder = None
            statement_folder = None
            statement_date = None

            if len(parts) >= 4:
                year_str = parts[-4]
                month_folder = parts[-3]
                statement_folder = parts[-2]

                if year_str.isdigit():
                    year = int(year_str)

                m = re.match(r"(\d{2})-(\d{2})-(\d{2})$", statement_folder)
                if m:
                    dd, mm, yy = map(int, m.groups())
                    statement_date = datetime(2000 + yy, mm, dd).date()

            rows.append({
                "filepath": filepath,
                "filetype": os.path.splitext(filepath)[1].lower().replace(".", ""),
                "year": year,
                "month_folder": month_folder,
                "statement_folder": statement_folder,
                "statement_date": statement_date
            })

    df = pd.DataFrame(rows)
    df["statement_date"] = pd.to_datetime(df["statement_date"], errors="coerce")
    df["month"] = df["statement_date"].dt.month
    df["quarter"] = df["statement_date"].dt.quarter
    return df


# -----------------------------
# 2) REPORT 1 GENERATOR (YOU plug in Task 1 logic here)
# -----------------------------
def generate_report_1(transactions_root: str, fx_file: str, year: int, output_path: str):
    """
    This function must generate the SAME Excel report you created in Task 1,
    but filtered to the selected year.

    Inputs:
      - transactions_root: folder that contains 2024/2025
      - fx_file: path to FX Rates Excel
      - year: selected year (e.g., 2024)
      - output_path: full path where the Excel should be saved
    """

    # 1) Build index + filter year
    index_df = build_file_index(transactions_root)
    year_df = index_df[index_df["year"] == year].copy()

    if year_df.empty:
        raise ValueError(f"No files found for year {year} in: {transactions_root}")

    # 2) Load transactions from all files for that year
    all_frames = []
    for fp in year_df["filepath"]:
        if fp.lower().endswith(".csv"):
    try:
        df = pd.read_csv(fp, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(fp, encoding="latin1")

    # If a file has a bad row, skip it but keep going
    # (this is the most practical fix for messy exports)
    df = pd.read_csv(
        fp,
        engine="python",       # more tolerant parser
        on_bad_lines="skip"    # skips problematic lines
    )
else:
    df = pd.read_excel(fp)
    tx = pd.concat(all_frames, ignore_index=True)

    # 3) Add metadata from index (OPTIONAL, but good)
    #    If your Task 1 report relied on statement_date/month/quarter/source_id,
    #    you should merge it properly here.
    #    For now we keep it simple.

    # 4) Load FX table (you already did this in Task 1)
    fx_raw = pd.read_excel(fx_file, header=None)

    # Find header row where "COUNTRY" appears (same as you did)
    header_row = None
    for i in range(len(fx_raw)):
        if fx_raw.iloc[i].astype(str).str.contains("COUNTRY", case=False, na=False).any():
            header_row = i
            break

    if header_row is None:
        raise ValueError("Could not find COUNTRY header in FX file.")

    fx = pd.read_excel(fx_file, skiprows=header_row)
    fx.columns = [c.strip().upper() for c in fx.columns]
    fx = fx[["COUNTRY", "CURRENCY", "CODE", "RATE"]].dropna(subset=["CODE", "RATE"])
    fx["CODE"] = fx["CODE"].astype(str).str.upper()

    # 5) Convert transactions to USD (adjust column names as needed)
    #    IMPORTANT: your dataset columns must match. Example you had: Currency, Transaction
    tx["Currency"] = tx["Currency"].astype(str).str.upper()

    tx = tx.merge(fx[["CODE", "RATE"]], left_on="Currency", right_on="CODE", how="left")
    if tx["RATE"].isna().any():
        missing = tx.loc[tx["RATE"].isna(), "Currency"].unique().tolist()
        raise ValueError(f"Missing FX rates for: {missing}")

    tx["Transaction_USD"] = tx["Transaction"] * tx["RATE"]

    # 6) Create the SAME summaries you used in Task 1
    #    A) total transactions per client per currency
    client_currency = (
        tx.groupby(["Client", "Currency"], as_index=False)
          .agg(
              transaction_count=("Transaction", "count"),
              total_local_amount=("Transaction", "sum"),
              total_usd_amount=("Transaction_USD", "sum")
          )
          .sort_values(["total_usd_amount"], ascending=False)
    )

    #    B) totals by client in USD
    client_usd = (
        tx.groupby("Client", as_index=False)
          .agg(
              transaction_count=("Transaction", "count"),
              total_usd_amount=("Transaction_USD", "sum")
          )
          .sort_values("total_usd_amount", ascending=False)
    )

    # 7) Export to Excel (Task 1 report)
    # Use openpyxl (usually installed with Anaconda)
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        client_usd.to_excel(writer, sheet_name="Client_USD_Totals", index=False)
        client_currency.to_excel(writer, sheet_name="Client_Currency_Totals", index=False)
        tx.to_excel(writer, sheet_name="Transactions_Detail", index=False)

    return output_path


# -----------------------------
# 3) GUI APP
# -----------------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("FP&A Report Generator (Task 4)")
        self.geometry("820x420")
        self.resizable(False, False)

        self.transactions_root = None
        self.fx_path = None
        self.index_df = None

        self.selected_folder_var = tk.StringVar(value="No folder selected yet.")
        self.fx_file_var = tk.StringVar(value="No FX file selected yet.")
        self.status = tk.StringVar(value="Step 1: Pick Transactions Folder")

        # --- Transactions folder ---
        tk.Label(self, text="Transactions Root Folder (contains 2024/2025):").pack(anchor="w", padx=12, pady=(12, 2))
        tk.Entry(self, textvariable=self.selected_folder_var, width=115).pack(padx=12)
        tk.Button(self, text="Pick Transactions Folder", command=self.pick_transactions_folder, height=2).pack(pady=8)

        # --- FX file ---
        tk.Label(self, text="FX Rates File (Excel):").pack(anchor="w", padx=12, pady=(10, 2))
        tk.Entry(self, textvariable=self.fx_file_var, width=115).pack(padx=12)
        tk.Button(self, text="Pick FX File", command=self.pick_fx_file, height=2).pack(pady=8)

        # --- Year dropdown ---
        tk.Label(self, text="Select Year:").pack(anchor="w", padx=12, pady=(10, 2))
        self.year_var = tk.StringVar(value="")
        self.year_dropdown = tk.OptionMenu(self, self.year_var, "")
        self.year_dropdown.config(width=20)
        self.year_dropdown.pack(anchor="w", padx=12)

        # --- Generate button ---
        tk.Button(self, text="Generate Report 1 (Task 1)", command=self.generate_report, height=2, width=30).pack(pady=14)

        # --- Status ---
        tk.Label(self, textvariable=self.status, fg="navy", wraplength=780, justify="left").pack(padx=12, pady=10)

    def pick_transactions_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select folder that contains 2024/2025")
            if not folder:
                self.status.set("No folder selected.")
                return

            self.transactions_root = folder
            self.selected_folder_var.set(folder)

            self.status.set("Scanning files...")
            self.update_idletasks()

            df = build_file_index(folder)
            self.index_df = df

            years = sorted(int(y) for y in df["year"].dropna().unique())
            if not years:
                self.status.set("❌ No years detected. Select the folder ABOVE the year folder (contains 2024/2025).")
                return

            # Populate dropdown
            menu = self.year_dropdown["menu"]
            menu.delete(0, "end")
            for y in years:
                menu.add_command(label=str(y), command=lambda v=y: self.year_var.set(str(v)))

            self.year_var.set(str(years[0]))

            self.status.set(f"✅ Files found: {len(df)} | Years detected: {years}")

            messagebox.showinfo("Scan Complete", f"Files found: {len(df)}\nYears detected: {years}")

        except Exception:
            self.status.set("❌ Error scanning folder. See console.")
            print(traceback.format_exc())

    def pick_fx_file(self):
        fx = filedialog.askopenfilename(
            title="Select FX Rates Excel file",
            filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")]
        )
        if fx:
            self.fx_path = fx
            self.fx_file_var.set(fx)
            self.status.set("✅ FX file selected.")

    def generate_report(self):
        try:
            if not self.transactions_root:
                messagebox.showwarning("Missing Input", "Please pick the Transactions folder first.")
                return
            if not self.fx_path:
                messagebox.showwarning("Missing Input", "Please pick the FX file.")
                return
            if not self.year_var.get():
                messagebox.showwarning("Missing Input", "Please select a year.")
                return

            year = int(self.year_var.get())

            # Choose output location
            default_name = f"Report_1_FPA_Analysis_{year}.xlsx"
            output_path = filedialog.asksaveasfilename(
                title="Save Report 1 Excel",
                defaultextension=".xlsx",
                initialfile=default_name,
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not output_path:
                self.status.set("Cancelled saving.")
                return

            self.status.set("Generating Report 1... please wait.")
            self.update_idletasks()

            saved_path = generate_report_1(
                transactions_root=self.transactions_root,
                fx_file=self.fx_path,
                year=year,
                output_path=output_path
            )

            self.status.set(f"✅ Report generated: {saved_path}")
            messagebox.showinfo("Success", f"Report 1 created successfully:\n\n{saved_path}")

        except Exception as e:
            self.status.set("❌ Failed generating report. See error popup.")
            messagebox.showerror("Error generating report", str(e))
            print(traceback.format_exc())


if __name__ == "__main__":
    App().mainloop()


IndentationError: expected an indented block after 'if' statement on line 88 (3526780422.py, line 89)

In [6]:
import os
import re
import traceback
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox

# -----------------------------
# 1) Build file index
# -----------------------------
def build_file_index(base_dir: str) -> pd.DataFrame:
    rows = []

    for root, _, files in os.walk(base_dir):
        for f in files:
            f_low = f.lower()

            # Only transaction files
            if "datatrans" not in f_low:
                continue
            if not f_low.endswith((".csv", ".xlsx")):
                continue

            filepath = os.path.join(root, f)

            # expects .../<YEAR>/<MONTH>/<DD-MM-YY>/DataTrans_...
            parts = os.path.normpath(filepath).split(os.sep)

            year = None
            month_folder = None
            statement_folder = None
            statement_date = None

            if len(parts) >= 4:
                year_str = parts[-4]
                month_folder = parts[-3]
                statement_folder = parts[-2]

                if year_str.isdigit():
                    year = int(year_str)

                m = re.match(r"(\d{2})-(\d{2})-(\d{2})$", statement_folder)
                if m:
                    dd, mm, yy = map(int, m.groups())
                    statement_date = datetime(2000 + yy, mm, dd).date()

            rows.append({
                "filepath": filepath,
                "filetype": os.path.splitext(filepath)[1].lower().replace(".", ""),
                "year": year,
                "month_folder": month_folder,
                "statement_folder": statement_folder,
                "statement_date": statement_date
            })

    df = pd.DataFrame(rows)
    df["statement_date"] = pd.to_datetime(df["statement_date"], errors="coerce")
    df["month"] = df["statement_date"].dt.month
    df["quarter"] = df["statement_date"].dt.quarter
    return df


# -----------------------------
# 2) Robust file loaders
# -----------------------------
def read_transactions_file(fp: str) -> pd.DataFrame:
    """
    Reads CSV/XLSX safely.
    - CSV: tolerant parser, skips bad lines, tries UTF-8 then latin1
    """
    print(f"Reading file: {fp}")

    if fp.lower().endswith(".csv"):
        # Try UTF-8 first
        try:
            return pd.read_csv(
                fp,
                engine="python",
                on_bad_lines="skip",
                encoding="utf-8"
            )
        except UnicodeDecodeError:
            return pd.read_csv(
                fp,
                engine="python",
                on_bad_lines="skip",
                encoding="latin1"
            )

    # Excel
    return pd.read_excel(fp)


def load_fx_table(fx_file: str) -> pd.DataFrame:
    """
    Loads FX file, finds the header row where COUNTRY appears,
    then returns columns: COUNTRY, CURRENCY, CODE, RATE (CODE uppercase).
    """
    fx_raw = pd.read_excel(fx_file, header=None)

    header_row = None
    for i in range(len(fx_raw)):
        row_vals = fx_raw.iloc[i].astype(str)
        if row_vals.str.contains("COUNTRY", case=False, na=False).any():
            header_row = i
            break

    if header_row is None:
        raise ValueError("Could not find the FX header row containing 'COUNTRY'.")

    fx = pd.read_excel(fx_file, skiprows=header_row)
    fx.columns = [str(c).strip().upper() for c in fx.columns]

    required = {"COUNTRY", "CURRENCY", "CODE", "RATE"}
    if not required.issubset(set(fx.columns)):
        raise ValueError(f"FX table missing columns. Found: {list(fx.columns)}")

    fx = fx[list(required)].dropna(subset=["CODE", "RATE"])
    fx["CODE"] = fx["CODE"].astype(str).str.upper().str.strip()
    fx["RATE"] = pd.to_numeric(fx["RATE"], errors="coerce")
    fx = fx.dropna(subset=["RATE"])

    return fx


# -----------------------------
# 3) REPORT 1 generator (Task 1)
# -----------------------------
def generate_report_1(transactions_root: str, fx_file: str, year: int, output_path: str) -> str:
    # Build index and filter year
    index_df = build_file_index(transactions_root)
    year_df = index_df[index_df["year"] == year].copy()

    if year_df.empty:
        raise ValueError(f"No files found for year {year} in: {transactions_root}")

    # Load all transactions
    frames = []
    for fp in year_df["filepath"]:
        df = read_transactions_file(fp)
        df["source_file"] = os.path.basename(fp)
        frames.append(df)

    tx = pd.concat(frames, ignore_index=True)

    # Basic sanity check: required columns
    needed_cols = {"Client", "Country", "Currency", "Transaction"}
    if not needed_cols.issubset(set(tx.columns)):
        raise ValueError(
            f"Transactions missing required columns. Needed {needed_cols} but found {set(tx.columns)}"
        )

    # Clean column formats
    tx["Currency"] = tx["Currency"].astype(str).str.upper().str.strip()

    # Load FX
    fx = load_fx_table(fx_file)

    # Merge FX
    tx = tx.merge(fx[["CODE", "RATE"]], left_on="Currency", right_on="CODE", how="left")

    if tx["RATE"].isna().any():
        missing = sorted(tx.loc[tx["RATE"].isna(), "Currency"].unique().tolist())
        raise ValueError(f"Missing FX rates for currencies: {missing}")

    # Convert to USD
    tx["Transaction"] = pd.to_numeric(tx["Transaction"], errors="coerce")
    tx["Transaction_USD"] = tx["Transaction"] * tx["RATE"]

    # Summaries (Task 1)
    client_currency = (
        tx.groupby(["Client", "Currency"], as_index=False)
          .agg(
              transaction_count=("Transaction", "count"),
              total_local_amount=("Transaction", "sum"),
              total_usd_amount=("Transaction_USD", "sum")
          )
          .sort_values(["total_usd_amount"], ascending=False)
    )

    client_usd = (
        tx.groupby("Client", as_index=False)
          .agg(
              transaction_count=("Transaction", "count"),
              total_usd_amount=("Transaction_USD", "sum")
          )
          .sort_values("total_usd_amount", ascending=False)
    )

    # Export Excel (openpyxl = default with Anaconda)
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        client_usd.to_excel(writer, sheet_name="Client_USD_Totals", index=False)
        client_currency.to_excel(writer, sheet_name="Client_Currency_Totals", index=False)
        tx.to_excel(writer, sheet_name="Transactions_Detail", index=False)

    return output_path


# -----------------------------
# 4) GUI App
# -----------------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("FP&A Report Generator (Task 4)")
        self.geometry("840x450")
        self.resizable(False, False)

        self.transactions_root = None
        self.fx_path = None
        self.index_df = None

        self.selected_folder_var = tk.StringVar(value="No folder selected yet.")
        self.fx_file_var = tk.StringVar(value="No FX file selected yet.")
        self.status = tk.StringVar(value="Step 1: Pick Transactions Folder")

        # Transactions folder
        tk.Label(self, text="Transactions Root Folder (contains 2024/2025):").pack(anchor="w", padx=12, pady=(12, 2))
        tk.Entry(self, textvariable=self.selected_folder_var, width=115).pack(padx=12)
        tk.Button(self, text="Pick Transactions Folder", command=self.pick_transactions_folder, height=2).pack(pady=8)

        # FX file
        tk.Label(self, text="FX Rates File (Excel):").pack(anchor="w", padx=12, pady=(10, 2))
        tk.Entry(self, textvariable=self.fx_file_var, width=115).pack(padx=12)
        tk.Button(self, text="Pick FX File", command=self.pick_fx_file, height=2).pack(pady=8)

        # Year dropdown
        tk.Label(self, text="Select Year:").pack(anchor="w", padx=12, pady=(10, 2))
        self.year_var = tk.StringVar(value="")
        self.year_dropdown = tk.OptionMenu(self, self.year_var, "")
        self.year_dropdown.config(width=20)
        self.year_dropdown.pack(anchor="w", padx=12)

        # Generate button
        tk.Button(self, text="Generate Report 1 (Task 1)", command=self.generate_report, height=2, width=32).pack(pady=14)

        # Status
        tk.Label(self, textvariable=self.status, fg="navy", wraplength=800, justify="left").pack(padx=12, pady=10)

    def pick_transactions_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select folder that contains 2024/2025")
            if not folder:
                self.status.set("No folder selected.")
                return

            self.transactions_root = folder
            self.selected_folder_var.set(folder)

            self.status.set("Scanning files...")
            self.update_idletasks()

            df = build_file_index(folder)
            self.index_df = df

            years = sorted(int(y) for y in df["year"].dropna().unique())
            if not years:
                self.status.set("❌ No years detected. Select the folder ABOVE the year folder (contains 2024/2025).")
                return

            # Populate dropdown
            menu = self.year_dropdown["menu"]
            menu.delete(0, "end")
            for y in years:
                menu.add_command(label=str(y), command=lambda v=y: self.year_var.set(str(v)))

            self.year_var.set(str(years[0]))

            self.status.set(f"✅ Files found: {len(df)} | Years detected: {years}")
            messagebox.showinfo("Scan Complete", f"Files found: {len(df)}\nYears detected: {years}")

        except Exception:
            self.status.set("❌ Error scanning folder. See console.")
            print(traceback.format_exc())

    def pick_fx_file(self):
        fx = filedialog.askopenfilename(
            title="Select FX Rates Excel file",
            filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")]
        )
        if fx:
            self.fx_path = fx
            self.fx_file_var.set(fx)
            self.status.set("✅ FX file selected.")

    def generate_report(self):
        try:
            if not self.transactions_root:
                messagebox.showwarning("Missing Input", "Please pick the Transactions folder first.")
                return
            if not self.fx_path:
                messagebox.showwarning("Missing Input", "Please pick the FX file.")
                return
            if not self.year_var.get():
                messagebox.showwarning("Missing Input", "Please select a year.")
                return

            year = int(self.year_var.get())

            default_name = f"Report_1_FPA_Analysis_{year}.xlsx"
            output_path = filedialog.asksaveasfilename(
                title="Save Report 1 Excel",
                defaultextension=".xlsx",
                initialfile=default_name,
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not output_path:
                self.status.set("Cancelled saving.")
                return

            self.status.set("Generating Report 1... please wait.")
            self.update_idletasks()

            saved_path = generate_report_1(
                transactions_root=self.transactions_root,
                fx_file=self.fx_path,
                year=year,
                output_path=output_path
            )

            self.status.set(f"✅ Report generated: {saved_path}")
            messagebox.showinfo("Success", f"Report 1 created successfully:\n\n{saved_path}")

        except Exception as e:
            self.status.set("❌ Failed generating report. See error popup.")
            messagebox.showerror("Error generating report", str(e))
            print(traceback.format_exc())


if __name__ == "__main__":
    App().mainloop()


Reading file: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\04-01-24\DataTrans_0.csv
Reading file: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\04-01-24\DataTrans_1.xlsx
Reading file: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_0.csv
Reading file: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_1.csv
Reading file: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_2.csv
Reading file: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_3.csv
Reading file: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\26-01-24\DataTrans_0.cs

In [7]:
import os
import re
import traceback
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox

# ==========================================================
# 1) BUILD FILE INDEX
# ==========================================================
def build_file_index(base_dir: str) -> pd.DataFrame:
    rows = []

    for root, _, files in os.walk(base_dir):
        for f in files:
            f_low = f.lower()

            if "datatrans" not in f_low:
                continue
            if not f_low.endswith((".csv", ".xlsx")):
                continue

            filepath = os.path.join(root, f)
            parts = os.path.normpath(filepath).split(os.sep)

            year = None
            month_folder = None
            statement_folder = None
            statement_date = None

            if len(parts) >= 4:
                year_str = parts[-4]
                month_folder = parts[-3]
                statement_folder = parts[-2]

                if year_str.isdigit():
                    year = int(year_str)

                m = re.match(r"(\d{2})-(\d{2})-(\d{2})$", statement_folder)
                if m:
                    dd, mm, yy = map(int, m.groups())
                    statement_date = datetime(2000 + yy, mm, dd).date()

            rows.append({
                "filepath": filepath,
                "filetype": os.path.splitext(filepath)[1].lower().replace(".", ""),
                "year": year,
                "month_folder": month_folder,
                "statement_folder": statement_folder,
                "statement_date": statement_date
            })

    df = pd.DataFrame(rows)
    df["statement_date"] = pd.to_datetime(df["statement_date"], errors="coerce")
    df["month"] = df["statement_date"].dt.month
    df["quarter"] = df["statement_date"].dt.quarter

    return df


# ==========================================================
# 2) SAFE FILE READER
# ==========================================================
def read_transactions_file(fp: str) -> pd.DataFrame:
    print(f"Reading: {fp}")

    if fp.lower().endswith(".csv"):
        try:
            return pd.read_csv(
                fp,
                engine="python",
                on_bad_lines="skip",
                encoding="utf-8"
            )
        except UnicodeDecodeError:
            return pd.read_csv(
                fp,
                engine="python",
                on_bad_lines="skip",
                encoding="latin1"
            )
    else:
        return pd.read_excel(fp)


# ==========================================================
# 3) LOAD FX TABLE
# ==========================================================
def load_fx_table(fx_file: str) -> pd.DataFrame:
    fx_raw = pd.read_excel(fx_file, header=None)

    header_row = None
    for i in range(len(fx_raw)):
        if fx_raw.iloc[i].astype(str).str.contains("COUNTRY", case=False, na=False).any():
            header_row = i
            break

    if header_row is None:
        raise ValueError("FX file header not found (COUNTRY row missing).")

    fx = pd.read_excel(fx_file, skiprows=header_row)
    fx.columns = [str(c).strip().upper() for c in fx.columns]

    required = {"COUNTRY", "CURRENCY", "CODE", "RATE"}
    if not required.issubset(set(fx.columns)):
        raise ValueError(f"FX missing required columns. Found: {list(fx.columns)}")

    fx = fx[list(required)].dropna(subset=["CODE", "RATE"])
    fx["CODE"] = fx["CODE"].astype(str).str.upper().str.strip()
    fx["RATE"] = pd.to_numeric(fx["RATE"], errors="coerce")
    fx = fx.dropna(subset=["RATE"])

    return fx


# ==========================================================
# 4) GENERATE REPORT 1 (TASK 1)
# ==========================================================
def generate_report_1(transactions_root: str, fx_file: str, year: int, output_path: str):

    index_df = build_file_index(transactions_root)
    year_df = index_df[index_df["year"] == year]

    if year_df.empty:
        raise ValueError(f"No files found for year {year}")

    frames = []
    for fp in year_df["filepath"]:
        df = read_transactions_file(fp)
        df["source_file"] = os.path.basename(fp)
        frames.append(df)

    tx = pd.concat(frames, ignore_index=True)

    # Validate required columns
    required_cols = {"Client", "Country", "Currency", "Transaction"}
    if not required_cols.issubset(tx.columns):
        raise ValueError(f"Missing required columns. Found: {tx.columns.tolist()}")

    # ------------------------------------------------------
    # FIX CURRENCY CLEANING (IMPORTANT FIX)
    # ------------------------------------------------------
    tx["Currency"] = tx["Currency"].astype("string").str.upper().str.strip()

    tx["Currency"] = tx["Currency"].replace(
        {"": pd.NA, "NAN": pd.NA, "NONE": pd.NA, "NULL": pd.NA, "NA": pd.NA}
    )

    missing_currency = tx["Currency"].isna().sum()
    if missing_currency > 0:
        print(f"Dropping {missing_currency} rows with missing currency.")
        tx = tx[tx["Currency"].notna()].copy()

    # ------------------------------------------------------

    # Load FX
    fx = load_fx_table(fx_file)

    # Merge FX
    tx = tx.merge(fx[["CODE", "RATE"]],
                  left_on="Currency",
                  right_on="CODE",
                  how="left")

    # Check missing FX after cleaning
    if tx["RATE"].isna().any():
        missing = sorted(tx.loc[tx["RATE"].isna(), "Currency"].unique())
        raise ValueError(f"Missing FX rates for currencies: {missing}")

    # Convert
    tx["Transaction"] = pd.to_numeric(tx["Transaction"], errors="coerce")
    tx["Transaction_USD"] = tx["Transaction"] * tx["RATE"]

    # ------------------------
    # Task 1 Outputs
    # ------------------------
    client_currency = (
        tx.groupby(["Client", "Currency"], as_index=False)
        .agg(
            transaction_count=("Transaction", "count"),
            total_local_amount=("Transaction", "sum"),
            total_usd_amount=("Transaction_USD", "sum")
        )
        .sort_values("total_usd_amount", ascending=False)
    )

    client_usd = (
        tx.groupby("Client", as_index=False)
        .agg(
            transaction_count=("Transaction", "count"),
            total_usd_amount=("Transaction_USD", "sum")
        )
        .sort_values("total_usd_amount", ascending=False)
    )

    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        client_usd.to_excel(writer, sheet_name="Client_USD_Totals", index=False)
        client_currency.to_excel(writer, sheet_name="Client_Currency_Totals", index=False)
        tx.to_excel(writer, sheet_name="Transactions_Detail", index=False)

    return output_path


# ==========================================================
# 5) GUI
# ==========================================================
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("FP&A Report Generator - Task 4")
        self.geometry("800x400")

        self.transactions_root = None
        self.fx_file = None
        self.year_var = tk.StringVar()

        tk.Button(self, text="Pick Transactions Folder",
                  command=self.pick_transactions).pack(pady=10)

        tk.Button(self, text="Pick FX File",
                  command=self.pick_fx).pack(pady=10)

        tk.Label(self, text="Select Year").pack()
        self.year_dropdown = tk.OptionMenu(self, self.year_var, "")
        self.year_dropdown.pack(pady=5)

        tk.Button(self, text="Generate Report 1",
                  command=self.generate_report).pack(pady=20)

    def pick_transactions(self):
        folder = filedialog.askdirectory()
        if not folder:
            return

        self.transactions_root = folder
        df = build_file_index(folder)
        years = sorted(int(y) for y in df["year"].dropna().unique())

        menu = self.year_dropdown["menu"]
        menu.delete(0, "end")

        for y in years:
            menu.add_command(label=str(y),
                             command=lambda v=y: self.year_var.set(str(v)))

        self.year_var.set(str(years[0]))

        messagebox.showinfo("Scan Complete",
                            f"Files found: {len(df)}\nYears detected: {years}")

    def pick_fx(self):
        fx = filedialog.askopenfilename(
            filetypes=[("Excel Files", "*.xlsx")]
        )
        if fx:
            self.fx_file = fx

    def generate_report(self):
        if not self.transactions_root or not self.fx_file:
            messagebox.showwarning("Missing Input",
                                   "Please select Transactions folder and FX file.")
            return

        year = int(self.year_var.get())

        output_path = filedialog.asksaveasfilename(
            defaultextension=".xlsx",
            initialfile=f"Report_1_FPA_Analysis_{year}.xlsx",
            filetypes=[("Excel Files", "*.xlsx")]
        )

        if not output_path:
            return

        try:
            saved = generate_report_1(
                self.transactions_root,
                self.fx_file,
                year,
                output_path
            )

            messagebox.showinfo("Success",
                                f"Report generated:\n{saved}")

        except Exception as e:
            messagebox.showerror("Error generating report", str(e))
            print(traceback.format_exc())


if __name__ == "__main__":
    App().mainloop()


Reading: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\04-01-24\DataTrans_0.csv
Reading: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\04-01-24\DataTrans_1.xlsx
Reading: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_0.csv
Reading: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_1.csv
Reading: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_2.csv
Reading: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\11-01-24\DataTrans_3.csv
Reading: C:/Users/ftorr/Dropbox/My PC (DESKTOP-9IFOLRB)/Documents/FPA/Transactions/Transacciones\2024\01 January\26-01-24\DataTrans_0.csv
Reading: C:/Users/ftorr/Dropbox/M